# BÁO CÁO ĐÁNH GIÁ DỮ LIỆU HUẤN LUYỆN VÀ TEST SET

# Hội thi Thử thách Trí tuệ Nhân tạo Thành phố Hồ Chí Minh 2024

## Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2024. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

## Cài đặt ban đầu

Bạn cần cài đặt môi trường để chạy được notebook này trên máy tính cá nhân của bạn. Hướng dẫn này không bao gồm phần cài đặt môi trường. Khuyến nghị: các bạn có thể cài đặt [Anaconda](https://docs.anaconda.com/free/anaconda/install/windows/).

## Cài đặt các thư viện FiftyOne và PyTorch
Hướng dẫn này dùng fiftyone là công cụ để trực quan dữ liệu và pytorch là backend chính cho các thuật toán máy học.


In [ ]:
# ! pip install fiftyone==0.24.1
# ! pip install torch torchvision torchaudio


Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os
from pathlib import Path



Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

## Directory

In [ ]:
parent_dir =  Path.cwd()
key_frame_dir = parent_dir / 'keyframe'
key_frame_dir

## Visualization

In [ ]:
dataset = fo.Dataset.from_images_dir(key_frame_dir, name=None, tags=None, recursive=True)

Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5151](http://localhost:5151)

In [ ]:
session = fo.launch_app(dataset, auto=False)

Hoặc bạn có thể chạy cell bên dưới để mở tab mới cho ứng dụng web fiftyone

In [ ]:
session.open_tab()

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [7]:
for sample in dataset:
    # print(sample['filepath'][:-4].rsplit('/', 2))
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('/', 2)
    sample.save()

Bạn có thể xem `Sample` đầu tiên của `Dataset` bằng lệnh sau:

In [ ]:
print(dataset.first())

### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [ ]:
# for sample in dataset:
#     object_path =  parent_dir / "objects-sample" / "objects" / str(sample['video']) / (sample['frameid'] + ".json")
#     print(object_path)
#     with open(object_path) as jsonfile:
#         det_data = json.load(jsonfile)
#     detections = []
#     for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
#         # Convert to [top-left-x, top-left-y, width, height]
#         boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
#         scoref = float(score)

#         # Only add objects with confidence > 0.4
#         if scoref > 0.4:
#             detections.append(
#                 fo.Detection(
#                     label=cls,
#                     bounding_box= boxf,
#                     confidence=float(score)
#                 )
#             )
#     sample["object_faster_rcnn"] = fo.Detections(detections=detections)
#     sample.save()


### Thêm thông tin CLIP embedding.

In [ ]:
all_keyframe = list(key_frame_dir.rglob('*.jpg'))
all_keyframe = [str(key) for key in all_keyframe]
video_keyframe_dict = {}
all_video = list(glob(str(key_frame_dir / "*")))
# print(all_video)
all_video = [v.rsplit('/',1)[-1] for v in all_video]
print( all_video)

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [ ]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('/',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)
video_keyframe_dict

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [11]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

In [ ]:
video_keyframe_dict

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [ ]:
embedding_dict = {}
for v in all_video:
    clip_path = list(parent_dir.glob(f"downloads/clip-features-32-b1/clip-features-32/{v}.npy"))
    a = np.array(np.load(str(clip_path[0])))
    print(type(a[0][0]))
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]


Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [27]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


In [ ]:
embedding_dict.keys()

In [ ]:
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="img_sim",
)

## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [ ]:
# Bạn cần phải cài version umap-learn hỗ trợ.
fob.compute_visualization(
    dataset,
    embeddings=clip_embeddings,
    brain_key="img_viz"
)
